# Library

In [1]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [17]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
from pyvis.network import Network as net
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")
from utils.Validator import *
from utils.Combinations import *
from utils.NetworkDraw import *
pd.set_option('display.max_columns', None)

# Network Followers

In [12]:
df_followers = pd.read_csv(r"../data/processed/DiGraph_Followers.csv")
df_followers.head()

,UserId,FollowerUserId
0,368,993
1,368,1902
2,368,1950
3,368,3258
4,368,3429


In [13]:
df_followers["UserId"] = df_followers["UserId"].astype(str)
df_followers["FollowerUserId"] = df_followers["FollowerUserId"].astype(str)
df_followers.shape

(765508, 2)

In [14]:
GUF = nx.from_pandas_edgelist(df_followers, source = "UserId", target = "FollowerUserId", create_using=nx.DiGraph())
print(nx.info(GUF))

Name: 
Type: DiGraph
Number of nodes: 253893
Number of edges: 765508
Average in degree:   3.0151
Average out degree:   3.0151


In [29]:
userId = "326814"

#Obtengo a los vecinos
predecessors = list(GUF.predecessors(userId))
successors = list(GUF.successors(userId))
list_color = []
for n in GUF.nodes:
    var = (0,0)
    if n == userId:
        var = (n,"#6EFAAC") # ALERTA
    elif n in predecessors or n in successors: #2DO NIVEL
        var = (n,"#4D82E3")
    else:
        var = (n,"#61C0FA") #3ER NIVEL
    list_color.append(var)
dict_color = dict(list_color)
nx.set_node_attributes(GUF, name='color', values=dict_color)

In [30]:
scaler = MinMaxScaler(feature_range=(20,100))
grouped = ["UserId"]
var_grouped = "FollowerUserId"
df_in = df_followers.groupby(grouped).agg({var_grouped: [(var_grouped,'count')]}).reset_index().copy()
df_in.columns = df_in.columns.droplevel(0)
df_in.columns = grouped + df_in.columns[1:].tolist()
df_in[var_grouped] = df_in[var_grouped].round(2)
#ESCALAMOS LA VARIABLE
df_in_scaled = scaler.fit_transform(df_in[var_grouped].values.reshape(-1,1))
df_in_scaled = pd.DataFrame(df_in_scaled,columns=[var_grouped])
df_in[var_grouped+"_SCALED"] = df_in_scaled[var_grouped]
df_in[var_grouped+"_SCALED"].fillna(10,inplace=True)

In [31]:
df_in

,UserId,FollowerUserId,FollowerUserId_SCALED
0,10000014,2,20.004751
1,100003,4,20.014253
2,1000076,8,20.033256
3,1000101,2,20.004751
4,1000173,2,20.004751
...,...,...,...
52155,999864,1,20.000000
52156,999866,5,20.019004
52157,99989,57,20.266049
52158,9998926,1,20.000000


In [32]:
#TAMANIO (A PARTIR DE LA CANTIDAD DE FOLLOWERS)
dict_size = dict(df_in[["UserId","FollowerUserId_SCALED"]].values.tolist())
nx.set_node_attributes(GUF, name='size', values=dict_size)

In [33]:
SG = NetworkDraw.get_subgraph_from_with_neighbors(GUF,userId)
print(nx.info(SG))

Name: 
Type: DiGraph
Number of nodes: 45
Number of edges: 208
Average in degree:   4.6222
Average out degree:   4.6222


In [34]:
NetworkDraw.draw_graph3(SG,notebook=False,directed=True,only_physics_buttons=True,height="100%",width="100%",output_filename=userId+'_2.html')